In [ ]:
import pandas as pd

### Check season 15/16:

In [48]:
# --- Load files ---
sf_df = pd.read_excel('New Assets Report - Gil-2025-07-06-23-49-50.xlsx')
robo_df = pd.read_excel('tickets_season_15.xlsx')

# --- Parse dates ---
sf_df["Product Date"] = pd.to_datetime(sf_df["Product Date"], errors="coerce")
robo_df["Product Date"] = pd.to_datetime(robo_df["Product Date"], errors="coerce")

# --- Filter SF by date range ---
start_date = "2015-09-01"
end_date = "2016-06-30"
sf_df = sf_df[
    (sf_df["Product Date"] >= start_date) &
    (sf_df["Product Date"] <= end_date)
]

# Check that you have data
print("SF rows after filtering:", len(sf_df))
print("Robo rows:", len(robo_df))

SF rows after filtering: 73544
Robo rows: 73544


In [49]:
# --- Count occurrences ---

sf_df["Product Date"] = sf_df["Product Date"].dt.tz_localize(None)
robo_df["Product Date"] = robo_df["Product Date"].dt.tz_localize(None)

# Count in Robo by Asset Name + Product Date
robo_counts = (
    robo_df.groupby(["Asset Name", "Product Date"])
    .size()
    .reset_index(name="Robo_Count")
)

# Count in SF by Asset Name + Product Date
sf_counts = (
    sf_df.groupby(["Asset Name", "Product Date"])
    .size()
    .reset_index(name="SF_Count")
)

# --- Merge counts ---
compare_counts = pd.merge(
    robo_counts,
    sf_counts,
    on=["Asset Name", "Product Date"],
    how="outer"
)

# Fill NaNs with 0
compare_counts["Robo_Count"] = compare_counts["Robo_Count"].fillna(0).astype(int)
compare_counts["SF_Count"] = compare_counts["SF_Count"].fillna(0).astype(int)

# Compute difference
compare_counts["Difference"] = compare_counts["Robo_Count"] - compare_counts["SF_Count"]

# Sort for clarity
compare_counts = compare_counts.sort_values(by=["Product Date", "Asset Name"])

# Show result
compare_counts

,Asset Name,Product Date,Robo_Count,SF_Count,Difference
133,הפועל ירושלים נגד הפועל חולון,2015-10-03,6,6,0
134,הפועל ירושלים נגד הפועל חולון,2015-10-04,6,6,0
135,הפועל ירושלים נגד הפועל חולון,2015-10-06,4,4,0
293,הפועל ירושלים נגד נפטונאס קלייפדה,2015-10-06,1,1,0
136,הפועל ירושלים נגד הפועל חולון,2015-10-07,827,827,0
...,...,...,...,...,...
114,הפועל ירושלים נגד הפועל אילת,2016-06-05,450,450,0
115,הפועל ירושלים נגד הפועל אילת,2016-06-06,171,171,0
252,הפועל ירושלים נגד מכבי ראשון לציון,2016-06-07,4484,4484,0
253,הפועל ירושלים נגד מכבי ראשון לציון,2016-06-08,912,912,0


### Check season 16/17:

In [21]:
# --- Load files ---
sf_df = pd.read_excel('New Assets Report - Gil-2025-07-07-00-00-01.xlsx')
robo_df = pd.read_excel('tickets_season_16.xlsx')

# --- Parse dates ---
sf_df["Product Date"] = pd.to_datetime(sf_df["Product Date"], errors="coerce")
robo_df["Product Date"] = pd.to_datetime(robo_df["Product Date"], errors="coerce")

# --- Filter SF by date range ---
start_date = "2016-09-01"
end_date = "2017-06-30"
sf_df = sf_df[
    (sf_df["Product Date"] >= start_date) &
    (sf_df["Product Date"] <= end_date)
]

# Check that you have data
print("SF rows after filtering:", len(sf_df))
print("Robo rows:", len(robo_df))

SF rows after filtering: 76551
Robo rows: 76551


In [22]:
# --- Count occurrences ---

sf_df["Product Date"] = sf_df["Product Date"].dt.tz_localize(None)
robo_df["Product Date"] = robo_df["Product Date"].dt.tz_localize(None)

# Count in Robo by Asset Name + Product Date
robo_counts = (
    robo_df.groupby(["Asset Name", "Product Date"])
    .size()
    .reset_index(name="Robo_Count")
)

# Count in SF by Asset Name + Product Date
sf_counts = (
    sf_df.groupby(["Asset Name", "Product Date"])
    .size()
    .reset_index(name="SF_Count")
)

# --- Merge counts ---
compare_counts = pd.merge(
    robo_counts,
    sf_counts,
    on=["Asset Name", "Product Date"],
    how="outer"
)

# Fill NaNs with 0
compare_counts["Robo_Count"] = compare_counts["Robo_Count"].fillna(0).astype(int)
compare_counts["SF_Count"] = compare_counts["SF_Count"].fillna(0).astype(int)

# Compute difference
compare_counts["Difference"] = compare_counts["Robo_Count"] - compare_counts["SF_Count"]

# Sort for clarity
compare_counts = compare_counts.sort_values(by=["Product Date", "Asset Name"])

# Show result
compare_counts

,Asset Name,Product Date,Robo_Count,SF_Count,Difference
17,הפועל ירושלים נגד בני הרצליה,2016-09-29,692,692,0
18,הפועל ירושלים נגד בני הרצליה,2016-10-03,34,34,0
380,הפועל ירושלים נגד עירוני נהריה,2016-10-03,1,1,0
19,הפועל ירושלים נגד בני הרצליה,2016-10-05,21,21,0
20,הפועל ירושלים נגד בני הרצליה,2016-10-06,543,543,0
...,...,...,...,...,...
418,הפועל ירושלים נגד עירוני נהריה,2017-06-02,1293,1293,0
419,הפועל ירושלים נגד עירוני נהריה,2017-06-05,460,460,0
420,הפועל ירושלים נגד עירוני נהריה,2017-06-06,977,977,0
421,הפועל ירושלים נגד עירוני נהריה,2017-06-07,998,998,0


### Check season 17/18:

In [23]:
# --- Load files ---
sf_df = pd.read_excel('New Assets Report - Gil-2025-07-07-00-02-19.xlsx')
robo_df = pd.read_excel('tickets_season_17.xlsx')

# --- Parse dates ---
sf_df["Product Date"] = pd.to_datetime(sf_df["Product Date"], errors="coerce")
robo_df["Product Date"] = pd.to_datetime(robo_df["Product Date"], errors="coerce")

# --- Filter SF by date range ---
start_date = "2017-09-01"
end_date = "2018-06-30"
sf_df = sf_df[
    (sf_df["Product Date"] >= start_date) &
    (sf_df["Product Date"] <= end_date)
]

# Check that you have data
print("SF rows after filtering:", len(sf_df))
print("Robo rows:", len(robo_df))

SF rows after filtering: 35946
Robo rows: 35946


In [24]:
# --- Count occurrences ---

sf_df["Product Date"] = sf_df["Product Date"].dt.tz_localize(None)
robo_df["Product Date"] = robo_df["Product Date"].dt.tz_localize(None)

# Count in Robo by Asset Name + Product Date
robo_counts = (
    robo_df.groupby(["Asset Name", "Product Date"])
    .size()
    .reset_index(name="Robo_Count")
)

# Count in SF by Asset Name + Product Date
sf_counts = (
    sf_df.groupby(["Asset Name", "Product Date"])
    .size()
    .reset_index(name="SF_Count")
)

# --- Merge counts ---
compare_counts = pd.merge(
    robo_counts,
    sf_counts,
    on=["Asset Name", "Product Date"],
    how="outer"
)

# Fill NaNs with 0
compare_counts["Robo_Count"] = compare_counts["Robo_Count"].fillna(0).astype(int)
compare_counts["SF_Count"] = compare_counts["SF_Count"].fillna(0).astype(int)

# Compute difference
compare_counts["Difference"] = compare_counts["Robo_Count"] - compare_counts["SF_Count"]

# Sort for clarity
compare_counts = compare_counts.sort_values(by=["Product Date", "Asset Name"])

# Show result
compare_counts

,Asset Name,Product Date,Robo_Count,SF_Count,Difference
12,הפועל ירושלים נגד בודוצ'נוסט,2017-09-27,2,2,0
183,הפועל ירושלים נגד מכבי ראשון לציון,2017-09-27,2,2,0
13,הפועל ירושלים נגד בודוצ'נוסט,2017-10-02,2,2,0
14,הפועל ירושלים נגד בודוצ'נוסט,2017-10-03,2,2,0
184,הפועל ירושלים נגד מכבי ראשון לציון,2017-10-03,45,45,0
...,...,...,...,...,...
109,הפועל ירושלים נגד הפועל גלבוע גליל,2018-05-30,286,286,0
110,הפועל ירושלים נגד הפועל גלבוע גליל,2018-05-31,10,10,0
111,הפועל ירושלים נגד הפועל גלבוע גליל,2018-06-01,31,31,0
112,הפועל ירושלים נגד הפועל גלבוע גליל,2018-06-02,334,334,0


### Check season 18/19:

In [25]:
# --- Load files ---
sf_df = pd.read_excel('New Assets Report - Gil-2025-07-07-00-03-56.xlsx')
robo_df = pd.read_excel('tickets_season_18.xlsx')

# --- Parse dates ---
sf_df["Product Date"] = pd.to_datetime(sf_df["Product Date"], errors="coerce")
robo_df["Product Date"] = pd.to_datetime(robo_df["Product Date"], errors="coerce")

# --- Filter SF by date range ---
start_date = "2018-09-01"
end_date = "2019-06-30"
sf_df = sf_df[
    (sf_df["Product Date"] >= start_date) &
    (sf_df["Product Date"] <= end_date)
]

# Check that you have data
print("SF rows after filtering:", len(sf_df))
print("Robo rows:", len(robo_df))

SF rows after filtering: 49246
Robo rows: 49246


In [26]:
# --- Count occurrences ---

sf_df["Product Date"] = sf_df["Product Date"].dt.tz_localize(None)
robo_df["Product Date"] = robo_df["Product Date"].dt.tz_localize(None)

# Count in Robo by Asset Name + Product Date
robo_counts = (
    robo_df.groupby(["Asset Name", "Product Date"])
    .size()
    .reset_index(name="Robo_Count")
)

# Count in SF by Asset Name + Product Date
sf_counts = (
    sf_df.groupby(["Asset Name", "Product Date"])
    .size()
    .reset_index(name="SF_Count")
)

# --- Merge counts ---
compare_counts = pd.merge(
    robo_counts,
    sf_counts,
    on=["Asset Name", "Product Date"],
    how="outer"
)

# Fill NaNs with 0
compare_counts["Robo_Count"] = compare_counts["Robo_Count"].fillna(0).astype(int)
compare_counts["SF_Count"] = compare_counts["SF_Count"].fillna(0).astype(int)

# Compute difference
compare_counts["Difference"] = compare_counts["Robo_Count"] - compare_counts["SF_Count"]

# Sort for clarity
compare_counts = compare_counts.sort_values(by=["Product Date", "Asset Name"])

# Show result
compare_counts

,Asset Name,Product Date,Robo_Count,SF_Count,Difference
233,הפועל ירושלים נגד מכבי ראשון לציון,2018-10-06,2,2,0
197,הפועל ירושלים נגד לייטקבליס,2018-10-07,2,2,0
234,הפועל ירושלים נגד מכבי ראשון לציון,2018-10-07,11,11,0
198,הפועל ירושלים נגד לייטקבליס,2018-10-08,5,5,0
235,הפועל ירושלים נגד מכבי ראשון לציון,2018-10-08,261,261,0
...,...,...,...,...,...
109,הפועל ירושלים נגד הפועל באר שבע,2019-05-25,8,8,0
110,הפועל ירושלים נגד הפועל באר שבע,2019-05-26,50,50,0
111,הפועל ירושלים נגד הפועל באר שבע,2019-05-27,127,127,0
112,הפועל ירושלים נגד הפועל באר שבע,2019-05-28,101,101,0


### Check season 19/20:

In [27]:
# --- Load files ---
sf_df = pd.read_excel('New Assets Report - Gil-2025-07-07-00-05-38.xlsx')
robo_df = pd.read_excel('tickets_season_19.xlsx')

# --- Parse dates ---
sf_df["Product Date"] = pd.to_datetime(sf_df["Product Date"], errors="coerce")
robo_df["Product Date"] = pd.to_datetime(robo_df["Product Date"], errors="coerce")

# --- Filter SF by date range ---
start_date = "2019-09-01"
end_date = "2020-06-30"
sf_df = sf_df[
    (sf_df["Product Date"] >= start_date) &
    (sf_df["Product Date"] <= end_date)
]

# Check that you have data
print("SF rows after filtering:", len(sf_df))
print("Robo rows:", len(robo_df))

SF rows after filtering: 40441
Robo rows: 40441


In [28]:
# --- Count occurrences ---

sf_df["Product Date"] = sf_df["Product Date"].dt.tz_localize(None)
robo_df["Product Date"] = robo_df["Product Date"].dt.tz_localize(None)

# Count in Robo by Asset Name + Product Date
robo_counts = (
    robo_df.groupby(["Asset Name", "Product Date"])
    .size()
    .reset_index(name="Robo_Count")
)

# Count in SF by Asset Name + Product Date
sf_counts = (
    sf_df.groupby(["Asset Name", "Product Date"])
    .size()
    .reset_index(name="SF_Count")
)

# --- Merge counts ---
compare_counts = pd.merge(
    robo_counts,
    sf_counts,
    on=["Asset Name", "Product Date"],
    how="outer"
)

# Fill NaNs with 0
compare_counts["Robo_Count"] = compare_counts["Robo_Count"].fillna(0).astype(int)
compare_counts["SF_Count"] = compare_counts["SF_Count"].fillna(0).astype(int)

# Compute difference
compare_counts["Difference"] = compare_counts["Robo_Count"] - compare_counts["SF_Count"]

# Sort for clarity
compare_counts = compare_counts.sort_values(by=["Product Date", "Asset Name"])

# Show result
compare_counts

,Asset Name,Product Date,Robo_Count,SF_Count,Difference
168,הפועל ירושלים נגד עירוני נהריה,2019-10-03,19,19,0
0,הפועל ירושלים נגד אאק אתונה,2019-10-04,1,1,0
169,הפועל ירושלים נגד עירוני נהריה,2019-10-04,5,5,0
1,הפועל ירושלים נגד אאק אתונה,2019-10-05,3,3,0
170,הפועל ירושלים נגד עירוני נהריה,2019-10-05,9,9,0
...,...,...,...,...,...
227,הפועל ירושלים נגד פרישטרי,2020-03-04,2167,2167,0
149,הפועל ירושלים נגד מכבי תל אביב,2020-03-05,206,206,0
150,הפועל ירושלים נגד מכבי תל אביב,2020-03-06,197,197,0
151,הפועל ירושלים נגד מכבי תל אביב,2020-03-07,32,32,0


### Check season 21/22:

In [29]:
# --- Load files ---
sf_df = pd.read_excel('New Assets Report - Gil-2025-07-07-00-07-31.xlsx')
robo_df = pd.read_excel('tickets_season_21.xlsx')

# --- Parse dates ---
sf_df["Product Date"] = pd.to_datetime(sf_df["Product Date"], errors="coerce")
robo_df["Product Date"] = pd.to_datetime(robo_df["Product Date"], errors="coerce")

# --- Filter SF by date range ---
start_date = "2021-09-01"
end_date = "2022-06-30"
sf_df = sf_df[
    (sf_df["Product Date"] >= start_date) &
    (sf_df["Product Date"] <= end_date)
]

# Check that you have data
print("SF rows after filtering:", len(sf_df))
print("Robo rows:", len(robo_df))

SF rows after filtering: 60610
Robo rows: 60610


In [30]:
# --- Count occurrences ---

sf_df["Product Date"] = sf_df["Product Date"].dt.tz_localize(None)
robo_df["Product Date"] = robo_df["Product Date"].dt.tz_localize(None)

# Count in Robo by Asset Name + Product Date
robo_counts = (
    robo_df.groupby(["Asset Name", "Product Date"])
    .size()
    .reset_index(name="Robo_Count")
)

# Count in SF by Asset Name + Product Date
sf_counts = (
    sf_df.groupby(["Asset Name", "Product Date"])
    .size()
    .reset_index(name="SF_Count")
)

# --- Merge counts ---
compare_counts = pd.merge(
    robo_counts,
    sf_counts,
    on=["Asset Name", "Product Date"],
    how="outer"
)

# Fill NaNs with 0
compare_counts["Robo_Count"] = compare_counts["Robo_Count"].fillna(0).astype(int)
compare_counts["SF_Count"] = compare_counts["SF_Count"].fillna(0).astype(int)

# Compute difference
compare_counts["Difference"] = compare_counts["Robo_Count"] - compare_counts["SF_Count"]

# Sort for clarity
compare_counts = compare_counts.sort_values(by=["Product Date", "Asset Name"])

# Show result
compare_counts

,Asset Name,Product Date,Robo_Count,SF_Count,Difference
160,הפועל ירושלים נגד פינאר קרשיאקה,2021-09-30,4,4,0
161,הפועל ירושלים נגד פינאר קרשיאקה,2021-10-01,3,3,0
162,הפועל ירושלים נגד פינאר קרשיאקה,2021-10-02,10,10,0
163,הפועל ירושלים נגד פינאר קרשיאקה,2021-10-03,962,962,0
164,הפועל ירושלים נגד פינאר קרשיאקה,2021-10-04,419,419,0
...,...,...,...,...,...
90,הפועל ירושלים נגד הפועל חולון,2022-05-26,1873,1873,0
91,הפועל ירושלים נגד הפועל חולון,2022-05-27,756,756,0
92,הפועל ירושלים נגד הפועל חולון,2022-05-31,2766,2766,0
93,הפועל ירושלים נגד הפועל חולון,2022-06-01,3013,3013,0


### Check season 22/23:

In [31]:
# --- Load files ---
sf_df = pd.read_excel('New Assets Report - Gil-2025-07-07-00-09-39.xlsx')
robo_df = pd.read_excel('tickets_season_22.xlsx')

# --- Parse dates ---
sf_df["Product Date"] = pd.to_datetime(sf_df["Product Date"], errors="coerce")
robo_df["Product Date"] = pd.to_datetime(robo_df["Product Date"], errors="coerce")

# --- Filter SF by date range ---
start_date = "2022-09-01"
end_date = "2023-06-30"
sf_df = sf_df[
    (sf_df["Product Date"] >= start_date) &
    (sf_df["Product Date"] <= end_date)
]

# Check that you have data
print("SF rows after filtering:", len(sf_df))
print("Robo rows:", len(robo_df))

SF rows after filtering: 77232
Robo rows: 77232


In [51]:
# --- Count occurrences ---

sf_df["Product Date"] = sf_df["Product Date"].dt.tz_localize(None)
robo_df["Product Date"] = robo_df["Product Date"].dt.tz_localize(None)

# Count in Robo by Asset Name + Product Date
robo_counts = (
    robo_df.groupby(["Asset Name", "Product Date"])
    .size()
    .reset_index(name="Robo_Count")
)

# Count in SF by Asset Name + Product Date
sf_counts = (
    sf_df.groupby(["Asset Name", "Product Date"])
    .size()
    .reset_index(name="SF_Count")
)

# --- Merge counts ---
compare_counts = pd.merge(
    robo_counts,
    sf_counts,
    on=["Asset Name", "Product Date"],
    how="outer"
)

# Fill NaNs with 0
compare_counts["Robo_Count"] = compare_counts["Robo_Count"].fillna(0).astype(int)
compare_counts["SF_Count"] = compare_counts["SF_Count"].fillna(0).astype(int)

# Compute difference
compare_counts["Difference"] = compare_counts["Robo_Count"] - compare_counts["SF_Count"]

# Sort for clarity
compare_counts = compare_counts.sort_values(by=["Product Date", "Asset Name"])

# Show result
compare_counts

,Asset Name,Product Date,Robo_Count,SF_Count,Difference
133,הפועל ירושלים נגד הפועל חולון,2015-10-03,6,6,0
134,הפועל ירושלים נגד הפועל חולון,2015-10-04,6,6,0
135,הפועל ירושלים נגד הפועל חולון,2015-10-06,4,4,0
293,הפועל ירושלים נגד נפטונאס קלייפדה,2015-10-06,1,1,0
136,הפועל ירושלים נגד הפועל חולון,2015-10-07,827,827,0
...,...,...,...,...,...
114,הפועל ירושלים נגד הפועל אילת,2016-06-05,450,450,0
115,הפועל ירושלים נגד הפועל אילת,2016-06-06,171,171,0
252,הפועל ירושלים נגד מכבי ראשון לציון,2016-06-07,4484,4484,0
253,הפועל ירושלים נגד מכבי ראשון לציון,2016-06-08,912,912,0


### Check season 23/24:

In [52]:
# --- Load files ---
sf_df = pd.read_excel('New Assets Report - Gil-2025-07-07-00-11-48.xlsx')
robo_df = pd.read_excel('tickets_season_23.xlsx')

# --- Parse dates ---
sf_df["Product Date"] = pd.to_datetime(sf_df["Product Date"], errors="coerce")
robo_df["Product Date"] = pd.to_datetime(robo_df["Product Date"], errors="coerce")

# --- Filter SF by date range ---
start_date = "2023-09-01"
end_date = "2024-06-30"
sf_df = sf_df[
    (sf_df["Product Date"] >= start_date) &
    (sf_df["Product Date"] <= end_date)
]

# Check that you have data
print("SF rows after filtering:", len(sf_df))
print("Robo rows:", len(robo_df))

SF rows after filtering: 44492
Robo rows: 44526


In [53]:
print("Supposed to be 34 missing: ", len(robo_df)-len(sf_df))

Supposed to be 34 missing:  34


In [34]:
# --- Count occurrences ---

sf_df["Product Date"] = sf_df["Product Date"].dt.tz_localize(None)
robo_df["Product Date"] = robo_df["Product Date"].dt.tz_localize(None)

# Count in Robo by Asset Name + Product Date
robo_counts = (
    robo_df.groupby(["Asset Name", "Product Date"])
    .size()
    .reset_index(name="Robo_Count")
)

# Count in SF by Asset Name + Product Date
sf_counts = (
    sf_df.groupby(["Asset Name", "Product Date"])
    .size()
    .reset_index(name="SF_Count")
)

# --- Merge counts ---
compare_counts = pd.merge(
    robo_counts,
    sf_counts,
    on=["Asset Name", "Product Date"],
    how="outer"
)

# Fill NaNs with 0
compare_counts["Robo_Count"] = compare_counts["Robo_Count"].fillna(0).astype(int)
compare_counts["SF_Count"] = compare_counts["SF_Count"].fillna(0).astype(int)

# Compute difference
compare_counts["Difference"] = compare_counts["Robo_Count"] - compare_counts["SF_Count"]

# Sort for clarity
compare_counts = compare_counts.sort_values(by=["Product Date", "Asset Name"])

# Show result
compare_counts

,Asset Name,Product Date,Robo_Count,SF_Count,Difference
22,הפועל ירושלים נגד הפועל גליל עליון,2023-12-12,439,439,0
23,הפועל ירושלים נגד הפועל גליל עליון,2023-12-13,78,78,0
24,הפועל ירושלים נגד הפועל גליל עליון,2023-12-14,235,235,0
25,הפועל ירושלים נגד הפועל גליל עליון,2023-12-15,38,38,0
26,הפועל ירושלים נגד הפועל גליל עליון,2023-12-16,309,309,0
...,...,...,...,...,...
53,הפועל ירושלים נגד הפועל חולון,2024-06-02,523,523,0
113,חצי גמר משחק 2: הפועל ירושלים נגד מכבי תל אביב,2024-06-04,4171,4171,0
114,חצי גמר משחק 2: הפועל ירושלים נגד מכבי תל אביב,2024-06-05,1056,1056,0
115,חצי גמר משחק 2: הפועל ירושלים נגד מכבי תל אביב,2024-06-06,1685,1685,0
